In [10]:
pip install torch torchvision

  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled typing-extensions-4.1.1
Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install transformers



  Attempting uninstall: fsspec
    Found existing installation: fsspec 2022.2.0
    Uninstalling fsspec-2022.2.0:
      Successfully uninstalled fsspec-2022.2.0


In [1]:
import os
import re
import pdfminer.high_level
import torch
from transformers import AutoTokenizer, AutoModel
import math
# Function to read PDF and extract text
def extract_text_from_pdf(pdf_path):
    output = ""
    with open(pdf_path, 'rb') as f:
        output = pdfminer.high_level.extract_text(f)
    return output

# Function to preprocess text
def preprocess_text(text):
    # Remove special characters and extra spaces
    text = re.sub(r'\s+', ' ', text)
    return text

# Function to split text into chunks
def split_text_into_chunks(text, chunk_size=1000):
    chunks = []
    for i in range(0, len(text), chunk_size):
        chunks.append(text[i:i+chunk_size])
    return chunks

# Function to embed text using a pre-trained model
def embed_text(text):
    tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/paraphrase-MiniLM-L6-v2")
    model = AutoModel.from_pretrained("sentence-transformers/paraphrase-MiniLM-L6-v2")
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = torch.mean(outputs.last_hidden_state, dim=1).squeeze().tolist()
    return embeddings



# Function to compute cosine similarity
def cosine_similarity(embedding1, embedding2):
    dot_product = sum(a * b for a, b in zip(embedding1, embedding2))
    magnitude1 = math.sqrt(sum(a ** 2 for a in embedding1))
    magnitude2 = math.sqrt(sum(b ** 2 for b in embedding2))
    if magnitude1 == 0 or magnitude2 == 0:
        return 0  # Handle edge case when one of the embeddings is zero
    return dot_product / (magnitude1 * magnitude2)

# Function to search for the answer using cosine similarity
def search_answer(question, chunks, chunk_embeddings, threshold=0.5):
    question_embedding = embed_text(question)
    max_similarity = -1
    best_chunk = None
    for i, chunk_embedding in enumerate(chunk_embeddings):
        similarity = cosine_similarity(question_embedding, chunk_embedding)
        if similarity > max_similarity:
            max_similarity = similarity
            best_chunk = chunks[i]
    return best_chunk if max_similarity > threshold else "I don't know the answer."


# Example usage
if __name__ == "__main__":
    document_path = "document.pdf"
    question = input("Ask a question: ")

    # Read document and preprocess text
    text = extract_text_from_pdf(document_path)
    text = preprocess_text(text)

    # Split text into chunks
    chunks = split_text_into_chunks(text)

    # Embed each chunk
    chunk_embeddings = [embed_text(chunk) for chunk in chunks]

    # Search for the answer
    answer = search_answer(question, chunks, chunk_embeddings, threshold=0.3)
    print("Answer:", answer)


Ask a question: who is one minute manager?
Answer:  RENAUD, President & Chief Executive Officer, Jerrico, Inc. “I believe The One Minute Manager should be made ‘standard issue’ at all management development training programs from new managers’ school to advanced management 3 Kenneth Blanchard & Spenser Johnson – THE ONE MINUTE MANAGER training. It embodies (in an easy-to-read form) the fundamental principles of people management we are trying to instill in our management team. I have made it required reading for all our managers.” —DAVID HANNA, President GRiD Systems Corporation “Buying copies of The One Minute Manager is one of the best investments I’ve made in myself and in our managers.” —LOUIS P. NEEB, President Fast Food Division, W. R. Grace & Co. (formerly Chairman of the Board, Burger King Corp.) “Should you apply one-minute management? Yes!” “The One Minute Manager ... don’t miss it!” —WORKING WOMAN —MERV GRIFFIN “Our managers are using The One Minute Manager ’s practical meth